In [1]:
!pip install sqlalchemy==2.0.10
!pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [5]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [6]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [13]:
#task 1
%sql select distinct Launch_Site from SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [22]:
#2
%sql select * from SPACEXTBL where Launch_Site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [16]:
#3
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


In [17]:
#4
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version like 'F9 v1.1%';

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


In [26]:
%sql select min(Date) from SPACEXTBL where [Landing _Outcome] = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


min(Date)
01-05-2017


In [34]:
%sql select (select Booster_Version from SPACEXTBL where [Landing _Outcome] = 'Success (drone ship)'),(select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ > '4000' and PAYLOAD_MASS__KG_ < '6000');
            
            

 * sqlite:///my_data1.db
Done.


(select Booster_Version from SPACEXTBL where [Landing _Outcome] = 'Success (drone ship)'),(select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ > '4000' and PAYLOAD_MASS__KG_ < '6000')
F9 FT B1021.1,F9 v1.1


In [36]:
%sql select Mission_Outcome, count(*) from SPACEXTBL group by Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,count(*)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [53]:
%sql select * FROM SPACEXTBL where day(Date) = 'Friday;'

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such function: day
[SQL: select * FROM SPACEXTBL where day(Date) = 'Friday;']
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [38]:
%sql select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [41]:
%sql select substr(Date, 4, 2) as month, Date, [Landing _Outcome],Booster_Version, Launch_Site from SPACEXTBL where substr(Date,7,4)='2015';  

 * sqlite:///my_data1.db
Done.


month,Date,Landing _Outcome,Booster_Version,Launch_Site
01,10-01-2015,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
02,11-02-2015,Controlled (ocean),F9 v1.1 B1013,CCAFS LC-40
03,02-03-2015,No attempt,F9 v1.1 B1014,CCAFS LC-40
04,14-04-2015,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
04,27-04-2015,No attempt,F9 v1.1 B1016,CCAFS LC-40
06,28-06-2015,Precluded (drone ship),F9 v1.1 B1018,CCAFS LC-40
12,22-12-2015,Success (ground pad),F9 FT B1019,CCAFS LC-40


In [48]:
%sql select * from SPACEXTBL where [Landing _Outcome] like 'Success%'  ;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
22-12-2015,01:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)
08-04-2016,20:43:00,F9 FT B1021.1,CCAFS LC-40,SpaceX CRS-8,3136,LEO (ISS),NASA (CRS),Success,Success (drone ship)
06-05-2016,05:21:00,F9 FT B1022,CCAFS LC-40,JCSAT-14,4696,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
27-05-2016,21:39:00,F9 FT B1023.1,CCAFS LC-40,Thaicom 8,3100,GTO,Thaicom,Success,Success (drone ship)
18-07-2016,04:45:00,F9 FT B1025.1,CCAFS LC-40,SpaceX CRS-9,2257,LEO (ISS),NASA (CRS),Success,Success (ground pad)
14-08-2016,05:26:00,F9 FT B1026,CCAFS LC-40,JCSAT-16,4600,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
14-01-2017,17:54:00,F9 FT B1029.1,VAFB SLC-4E,Iridium NEXT 1,9600,Polar LEO,Iridium Communications,Success,Success (drone ship)
19-02-2017,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
30-03-2017,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
01-05-2017,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
